In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

# Fazit
**TODO: Einleitung des Fazit Kapitels**

## Visualisierung

Das Notebook, das im Folgenden geladen wird, beinhaltet Funktionen für die grafische Darstellung des aktuellen Zustands.

In [ ]:
%run ./01_einleitung.ipynb

In [ ]:
%run ./02_markov_decision_process.ipynb

In [ ]:
%run ./03_reinforcement_learning.ipynb
%run ./implementierung_gui.ipynb

Folgendermaßen kann dann eine Episode ausgeführt werden. Als Methoden für die Auswahl der Aktionen sind `random_action` und `q_learning_action` möglich.

In [ ]:
state = (((3, 0), 0, 2))
transport = Transport(state, action_method = q_learning_action, stepwise = True, visualize = True)
update_canvas(transport)

In [ ]:
episode = transport.state_list
episode

In [ ]:
mc_table = np.zeros([len(states), len(actions)])
count_state_action = np.zeros([len(states), len(actions)])

In [ ]:
def mc_function(state_list):
    states = [s for (s, _,_) in state_list]
    state_action_pairs = {(s, a) for (s, a,_) in state_list}
    for (state, action) in state_action_pairs:
        idx = states.index(state)
        value = 0
        for i in range (idx, len(state_list)):
            value += state_list[i][2]
        state_num = state_to_state_num(state)
        count_state_action[state_num][action] += 1
        mc_table[state_num][action] = (mc_table[state_num][action] + value)/count_state_action[state_num][action]

In [ ]:
def mc_learning(episodes):
    epsilon = 0.3
    
    for episode in range(episodes):
        state = states[random.randrange(0, len(states), 1)]
        done  = False
        state_list = []
        transport  = Transport(state, None, stepwise = False, visualize = False)
        
        while not done:
            state_num = state_to_state_num(transport.state)
            if random.uniform(0, 1) < epsilon:
                action = random.sample(possible_actions[transport.state[0]], 1)[0] # Aktionsbereich erkunden
            else:
                action = np.argmax(mc_table[state_num]) # Gelernte Werte ausnutzen
                
            reward     = transport.reward_function(action)
            transport.state = transport.transition_function(action)
            state_list.append((transport.state, action, reward))
            _, position_goods, position_goal = transport.state
            if position_goods == position_goal: done = True
        mc_function(state_list)
        print(str(round((episode + 1)/episodes*100, 1)) + '%', end='\r')

In [ ]:
mc_learning(100)

In [ ]:
count_state_action

## Evaluation
Um die Performance des trainierten Agenten beurteilen zu können, wird für eine große Menge an zufälligen Startzuständen einmal zur Auswahl der Aktionen der zufällige Algorithmus genutzt und einmal jener, der auf den erlernten Werten basiert:

In [ ]:
def evaluation(number_of_tests, action_method):
    total_value = 0
    total_steps = 0

    for i in range (number_of_tests):
        state = states[random.randrange(0, len(states), 1)]
        transport = Transport(state, action_method = action_method, stepwise = False, visualize = False)
        transport.transport_goods()

        total_value += transport.current_value
        total_steps += transport.steps
        print(round((i+1)/number_of_tests * 100, 1), '%', end='\r')

    print('Result after ' + str(number_of_tests) + ' tests:')
    print('Average value: ' + str(total_value/number_of_tests))
    print('Average number of steps: ' + str(total_steps/number_of_tests))
    print('Average reward per step: ' + str((total_value/number_of_tests)/(total_steps/number_of_tests)))

In [ ]:
#evaluation(2000, random_action)

Ergebnis nach 2000 Durchläufen ohne Reinforcement Learning:
* Average value: -1404.6395
* Average number of steps: 890.9995
* Average reward per step: -1.5764761933087503

In [ ]:
#evaluation(2000, q_learning_action)

Ergebnis nach 2000 Durchläufen mit Reinforcement Learning und 10.000 Trainingsdurchläufen für das Q-Learning:
* Average value: 1.2275
* Average number of steps: 12.8825
* Average reward per step: 0.0952843004075296

Das Ergebnis zeigt deutlich, dass sich das Trainieren gelohnt hat und sich die Anzahl der benötigten Schritte pro Episode deutlich reduziert hat. Schaut man sich die Episoden in der Visualisierung an, sieht man auch, dass der trainierte Agent meistens den optimalen, also kürzesten, Weg nimmt.

**TODO: Auswertung für Monte-Carlo und TD-Learning + Ausführlicheres Fazit**